In [1]:
import os
import sys
root = '/home/mark/Documents/code/mdrp'
sys.path.append(root)

import numpy as np
import pickle
from utils.nlp_mdrp import mdrp_prob
import cyipopt

from utils.convert import convert_chi
from utils.nlp_mdrp import mdrp_prob
from utils.helpers_chi import *


In [2]:
def main(parameters):
    # Setup the directory and logger    
    # Load the instance

    data_path = parameters['data_path']
    prob_setup = convert_chi(data_path,parameters)
    # add additional parameters
    prob_setup['max_rho'] =  parameters['max_rho']
    prob_setup['demand']  =  parameters['demand']
    prob_setup['scale']   =  parameters['scale']
    # SOLVE NLP
    mdrp = mdrp_prob(prob_setup)
    lb, ub, cl, cu = mdrp.get_bounds()
    x0 = mdrp.guess_init()
    nlp = cyipopt.Problem(
        n=len(x0),
        m=len(cl),
        problem_obj=mdrp,
        lb=lb,
        ub=ub,
        cl=cl,
        cu=cu
        )

    # nlp.addOption('derivative_test', 'second-order')
    nlp.add_option('mu_strategy', 'adaptive')
    nlp.add_option('tol', 1e-4)

    x, info = nlp.solve(x0)

    # Eval
    return x, mdrp

In [3]:
parameters = {}
parameters['seed'] = 0
parameters['data_path'] = '/home/mark/Documents/code/mdrp/chicago/all.p'
parameters['scale'] = False
parameters['demand'] = 1550 # exact is 1472 per hour
# parameters['scale'] = True
# parameters['demand'] = 0.99 # exact is 1472 per hour
parameters['cost'] = [50,40,25] # air, luxury, regular
parameters['Beta_max'] = 0.3
parameters['Beta_min'] = 0.1
N_values = ([100,100,900],[1,110,990])
parameters['mode_names'] = ['UAT','AV','V']
parameters['max_rho'] = [0.75,0.95,0.95]
parameters['alpha_scale'] = np.array([80, 60, 40])/40
# parameters['alpha_scale'] = np.array([100, 40.06, 33.89])/33.89
# parameters['alpha_scale'] = np.array([1,1,1])
# N_values = (np.array([8500,500,1100]),)
# N_values = ([100,1,1],)

In [4]:
# # CHECK FOR MAX DEMAND
# N_values = ([100,100,900],[1,110,990])

# for N, i in zip(N_values, range(len(N_values))):
#     # Assumes first is only cars
#     if i == 0:
#         parameters['max_rho'] = [0.75,0.95,0.95]
#     elif i == 1:
#         parameters['max_rho'] = [0,0.95,0.95]

#     parameters['N'] = N
#     # parameters['max_rho'] = [0.75,0.95,0.95]
#     # load the instance
#     data_path = parameters['data_path']
#     prob_setup = convert_chi(data_path,parameters)
#     # add additional parameters

#     # print(prob_setup['avg_dph'].mean())
#     prob_setup['max_rho'] =  parameters['max_rho']
#     prob_setup['demand']  =  parameters['demand']
#     prob_setup['scale']   =  parameters['scale']
#     # SOLVE NLP
#     mdrp = mdrp_prob(prob_setup)

#     print(mdrp.tot_demand)
# # print(prob_setup['q_time']*60)

In [5]:
# import pickle 

# results = []

# for N, i in zip(N_values, range(len(N_values))):
#     # Assumes first is only cars
#     if i == 0:
#         parameters['max_rho'] = [0.75,0.95,0.95]
#     elif i == 1:
#         parameters['max_rho'] = [0,0.95,0.95]

#     parameters['N'] = N
#     # load the instance
#     result = {}
#     print('Running iter')

#     sol, mdrp = main(parameters)
#     result['sol'] = sol
#     result['mdrp'] = mdrp
#     result['parameters'] = parameters
#     results.append(result)



In [6]:
# results_path = '/home/mark/Documents/code/mdrp/results/chicago.p'
# pickle.dump(results, open(results_path,'wb'))

In [7]:
results_path = '/home/mark/Documents/code/mdrp/results/chicago.p'
results = pickle.load(open(results_path,'rb'))

for i in range(len(results)):
    result = results[i]
    mdrp = result['mdrp']
    x = result['sol']
    flow = mdrp.vec_to_mat(x)
    lat = mdrp.get_latency(flow)
    # mask when drones are slowest
    print('solution')
    print(flow.sum().sum(), mdrp.tot_demand)
    mask = np.argmax(lat,axis=1)==0 
    flow[mask,0] = 0
    # check that the solution is still valid (nearly zero flow)
    print('rounded solution')
    print(flow.sum().sum(), mdrp.tot_demand)
    # update solution by removing drones on inefficient routes
    result['sol'] = mdrp.mat_to_vec(flow)
    # evaluate
    prices, min_prices = eval_sol(result)
    # break

    


solution
1550.0 1549.9999999999998
rounded solution
1549.9999802424754 1549.9999999999998


Problem Evlatuation
--------------------------------------------------------
Orders Demanded (order/hour): 1550.00
--------------------------------------------------------
Mode 		|UAT	|AV	|T	|total	|
----------------|-------|-------|-------|-------|
N (# veh.)	|100	|100	|900	|
Orders (%)	|8.67	|8.17	|83.16	|100.00	|
Utilization (%)	|0.75	|0.87	|0.94	|0.92	|
D Time(min.)	|22.52	|30.46	|34.95	|33.50	|
Order Price	|112.15	|42.50	|40.96	|47.26	|
Op. Cost ($/hr)	|5000	|3000	|18000	|26000	|
Profit ($/hr)	|10079	|2379	|34799	|47257	|--------------------------------------------------------
Minimum Price ($): 39.15
--------------------------------------------------------


In [8]:
# NASH CONDITION IS SATISFIED EXPLICITLY
# NOTE 1e-6 is used for floating poat error, where flow is rounded to 4th decimal
count = 0
flow = np.round(flow,4)
for i in range(mdrp.n):
    if not mask[i]:
        # drone to car r
        cost_drone = alpha_func(mdrp.alpha_0[i,0], mdrp.delta[i,0], flow[i,0])*prices[i,0] + lat[i,0]
        cost_car = alpha_func(mdrp.alpha_0[i,2], mdrp.delta[i,2], flow[i,0])*prices[i,2] + lat[i,2]
        if cost_drone - 1e-6 > cost_car:
            count+= 1
            print('1: ',cost_drone,cost_car,flow[i])
        # drone to car l
        cost_drone = alpha_func(mdrp.alpha_0[i,0], mdrp.delta[i,0], flow[i,0])*prices[i,0] + lat[i,0]
        cost_car = alpha_func(mdrp.alpha_0[i,2], mdrp.delta[i,2], flow[i,0])*prices[i,2] + lat[i,2]
        if cost_drone - 1e-6 > cost_car:
            count+= 1
            print('2: ',cost_drone,cost_car,flow[i])
        # car to drone r
        cost_drone = alpha_func(mdrp.alpha_0[i,0], mdrp.delta[i,0], flow[i].sum())*prices[i,0] + lat[i,0]
        cost_car = alpha_func(mdrp.alpha_0[i,2], mdrp.delta[i,2], flow[i].sum())*prices[i,2] + lat[i,2]
        if cost_car - 1e-6 > cost_drone:
            count+= 1
            print('3: ',cost_drone,cost_car,flow[i])
        # car to drone l
        cost_drone = alpha_func(mdrp.alpha_0[i,0], mdrp.delta[i,0], flow[i].sum()-flow[i,2])*prices[i,0] + lat[i,0]
        cost_car = alpha_func(mdrp.alpha_0[i,2], mdrp.delta[i,2], flow[i].sum()-flow[i,2])*prices[i,2] + lat[i,2]
        if cost_car - 1e-6 > cost_drone:
            count+= 1
            print('4: ',cost_drone,cost_car,flow[i])
print(count)

0


In [9]:
# NASH CONDITION SATSIFIED IMPLICITLY
count = 0
for i in range(mdrp.n):
    if not mask[i]:
        if flow[i,1] > 0: 
            # drone to car r
            alpha1_a1 = alpha_func(mdrp.alpha_0[i,0], mdrp.delta[i,0], flow[i,0])
            alpha3_a1 = alpha_func(mdrp.alpha_0[i,2], mdrp.delta[i,2], flow[i,0])
            alpha1_a2 = alpha_func(mdrp.alpha_0[i,0], mdrp.delta[i,0], flow[i].sum()-flow[i,-1])
            alpha3_a2 = alpha_func(mdrp.alpha_0[i,2], mdrp.delta[i,2], flow[i].sum()-flow[i,-1])

            left  = ((alpha1_a1/alpha3_a1)*prices[i,0]) - ((lat[i,2] - lat[i,0])/alpha3_a1)
            right = ((alpha1_a2/alpha3_a2)*prices[i,0]) - ((lat[i,2] - lat[i,0])/alpha3_a2)

            if not all(np.argsort([left, prices[i,2], right])==[0,1,2]):
        # if np.argsort([left, prices[i,2], right]) != [0,1,2]:
                print(i)
                print(flow[i])
                count += 1

print(count)

0
